# QENS data reduction

This notebook will guide you through the data reduction for the QENS experiment that you simulated with McStas yesterday.

The following is a basic outline of what this notebook will cover:

- Loading the NeXus files that contain the data
- Inspect/visualize the data contents
- How to convert the raw `time-of-flight` coordinate to something more useful ($\Delta E$)
- Write the results to file

In [ ]:
import numpy as np
import scipp as sc
import plopp as pp
import scippneutron as scn
import qens_utils as utils

## Process the elastic sample data

### Load raw data

Begin by loading and investigating the data obtained from the elastic sample.

In [ ]:
folder = "../3-mcstas/QENS_elastic_many_neutrons"

⚠️ **If you did not complete the QENS elastic simulation yesterday**,
you can use some pre-prepared data by uncommenting and running the cell below:

In [ ]:
# folder = utils.fetch_data("3-mcstas/QENS_elastic_many_neutrons")

In [ ]:
events = utils.load_qens(folder)

The first way to inspect the data is to view the HTML representation of the loaded object.

Try to explore what is inside the data, and familiarize yourself with the different sections (`Dimensions`, `Coordinates`, `Data`).

In [ ]:
events

### Visualize the raw data

Here is a histogram of the raw data in `tof`:

In [ ]:
events.hist(tof=100).plot()

It is more enlightening to look at a 2D histogram in `tof` and `y` as this shows a defect of the detector for a certain y-range:

In [ ]:
events.hist(tof=100, y=100).plot()

### Exercise 1: mask bad region

We need to handle the events that are affected by the detector defect.
The simplest way of doing so it to mask the affected region in y.
We will do so by binning in y and applying a y-dependent mask to the data array.

**Steps**:

1. Define bin-edges in the y-dimension that cover the entire range of the data.
There should be three bins which cover the y-range below the broken region, the broken region, and the the range above the broken region, respectively.
So you should end up with a length-4 array of bin-edges.
2. Define another array for the mask which should have the values `[False, True, False]`.
So it masks the middle bin but not the others.
3. Apply the mask by binning the events using the edges you defined and inserting the mask into the binned array's masks.

**Hints**:

- Use `sc.array` to create arrays for the bin-edges and the mask.
- Bin the data using `binned_for_mask = events.bin(...)`,
- Apply the mask by inserting it into `binned_for_mask.masks` which behaves like a `dict`.

**Solution:**

In [ ]:
y = events.coords["y"]
mask_regions = sc.array(
    dims=["y"],
    values=[0.21, 0.26, 0.27, 0.29],
    unit=y.unit,
)
binned_for_mask = events.bin(y=mask_regions)
mask = sc.array(dims=["y"], values=[False, True, False])
binned_for_mask.masks["bad_timing"] = mask
binned_for_mask

We plot the data array with the mask below.
The black band in the middle bin indicates the mask:

In [ ]:
binned_for_mask.hist().plot(vmin=0)

When making the same 2D histogram as before, the masked events will be dropped and the result is a histogram with a gap:

In [ ]:
binned_for_mask.hist(tof=100, y=100).plot()

### Transform to energy transfer

The next step is to compute the measured energy transfer in the sample from time-of-flight and the other coordinates of our data.
We use [scipp.transform_coords](https://scipp.github.io/generated/functions/scipp.transform_coords.html) for this purpose.

Scippneutron provides some pre-defined functions for coordinate transformations, see the [documentation](https://scipp.github.io/scippneutron/user-guide/coordinate-transformations.html) for a list.
In particular, it provides [scippneutron.conversion.tof.energy_transfer_indirect_from_tof](https://scipp.github.io/scippneutron/generated/modules/scippneutron.conversion.tof.energy_transfer_indirect_from_tof.html) which computes the energy transfer for an indirect-geometry spectrometer:

$$\Delta E = \frac{m_n L_1^2}{2 {(t-t_0)}^2} - \mathsf{final\_energy}\,, \qquad t_0 = \sqrt{\frac{L_2^2 m_n}{2 \mathsf{final\_energy}}}\,,$$

where $m_n$ is the neutron mass, $L_1$ is the primary flight path length (from source to sample), and $L_2$ is the secondary flight path length (from sample to detector).
The intermediate variable $t_0$ is the the time of flight for the secondary path such that $(t - t_0)$ is the arrival time at the sample.

While Scippneutron provides most of what we need, we have to define some custom components for our specific instrument.
We require functions to compute

- **L2**, the path length of the scattered neutron. The default function in Scippneutron assumes a straight path but here, we need to take the reflection from the analyzer crystal into account: $L_2 = |\overline{\mathsf{sample},\mathsf{analyzer}}| + |\overline{\mathsf{analyser},\mathsf{detector}}|$

- **final_wavelength**, the neutron wavelength that the analyzer selects for. We compute it from the known $d$-spacing of the analyzer $d_a$ and the scattering angle off the analyzer $\theta_a$: $\mathsf{final\_wavelength} = 2 d_a \sin{\theta_a}$

- **final_energy**, the energy of the neutrons that hit the detector (more below): $\mathsf{final\_energy} = \displaystyle\frac{m_n}{2} v_f^2$

The first two are implemented below.
They use the positions and analyzer parameters in the input data.

In [ ]:
def backscattered_l2(position, sample_position, analyzer_position):
    """
    Compute the length of the secondary flight path for backscattering off an analyzer.
    """
    return sc.norm(position - analyzer_position) + sc.norm(
        analyzer_position - sample_position
    )


def wavelength_from_analyzer(analyzer_dspacing, analyzer_angle):
    """
    Compute the neutron wavelength after scattering from the analyzer's d-spacing.

    Assuming Bragg scattering in the analyzer, the wavelength is
        wavelength = 2 * d * sin(theta)

    Where
        d is the analyzer's d-spacing,
        theta is the scattering angle or equivalently, the tilt of the analyzer
              w.r.t. to the sample-analyzer axis.
    """
    # 2*theta is the angle between transmitted and scattered beam.
    return (
        2
        * analyzer_dspacing
        * sc.sin(sc.scalar(np.pi / 2, unit="rad") - analyzer_angle.to(unit="rad"))
    )

We can start assembling the graph to compute the energy transfer:

In [ ]:
from scippneutron.conversion.graph.beamline import beamline
from scippneutron.conversion.tof import energy_transfer_indirect_from_tof

graph = {
    **beamline(scatter=True),
    "energy_transfer": energy_transfer_indirect_from_tof,
    # Replace L2 with our own implementation.
    "L2": backscattered_l2,
    # Insert a new function for the wavelength.
    "final_wavelength": wavelength_from_analyzer,
}
# Optional: remove unused functions in order to clean up the image below.
del graph["two_theta"]
del graph["scattered_beam"]
del graph["Ltotal"]
sc.show_graph(graph, simplified=True)

### Exercise 2: Compute energy transfer 

We can see that `final_energy` is not yet linked to `final_wavelength` in the graph.
Your task is to implement a function `def final_energy(final_wavelength)` to fill in the gap.

The energy is given by

$$\mathsf{final\_energy} = \frac{m_n}{2} v_f^2, \qquad v_f = \frac{2\pi\hbar}{m_n \mathsf{final\_wavelength}},$$

where $v_f$ is the speed of the scattered neutron and $m_n$ is the neutron mass.

#### Exercise 2.1: Finish the graph

**Steps**:

- Define a new function that computes the final energy given the equations above.
    - Tip: Ensure that the result has unit `meV`.
    - Tip: Use [sc.constants](https://scipp.github.io/generated/modules/scipp.constants.html) to get values for $\hbar$ (or $h$) and $m_n$.
- Insert the new function into the graph.

**Solution:**

In [ ]:
def final_energy(final_wavelength):
    """
    Compute the neutron energy after scattering.

    Uses
        final_energy = mn / 2 * final_speed**2
        final_speed = 2 * pi * hbar / mn / final_wavelength

    Where
        mn is the neutron mass,
        final_wavelength is the wavelength after scattering,
        final_speed is the speed after scattering.
    """
    return sc.to_unit(
        sc.constants.h**2 / 2 / sc.constants.neutron_mass / (final_wavelength**2),
        "meV",
    )


graph["final_energy"] = final_energy
sc.show_graph(graph, simplified=True)

#### Exercise 2.2: Compute energy transfer with the masked data

Use `binned_for_mask.transform_coords` (or what ever name you used for the masked array) to compute `"energy_transfer"`.

In [ ]:
in_energy_transfer = binned_for_mask.transform_coords("energy_transfer", graph=graph)
in_energy_transfer

#### Exercise 2.3: Compute energy transfer for unmasked data

For comparison, also compute energy transfer with the original data without a mask.

**Solution:**

In [ ]:
in_energy_transfer_unmasked = events.transform_coords("energy_transfer", graph=graph)

Now we can plot the masked and unmasked data to compare them.
As always, we first need to histogram them.
(We use `in_energy_transfer.bins.concat()` to undo the binning from exercise 1.
Without it, we would get a 2D histogram in `y` and `energy_transfer`.)

In [ ]:
masked_hist = in_energy_transfer.bins.concat().hist(energy_transfer=100)
unmasked_hist = in_energy_transfer_unmasked.hist(energy_transfer=100)

And finally, we can plot the data:

In [ ]:
pp.plot(
    {
        "unmasked": unmasked_hist,
        "masked": masked_hist,
    }
)

### Exercise 3: Compute energy transfer for all samples

We have only looked at one of our three samples so far.
Now, we repeat the same procedure for the remaining two.

Your task is to load the other samples, mask the broken detector region, and compute energy_transfer.
(You don't need to repeat the calculation for the unmasked data.

**Hints**:

- Write a function that encapsulates the whole procedure.
- You can reuse the mask bin-edges and the mask itself.
- Store the results for all samples in a `dict`. This way, you can easily plot it using `pp.plot`.
- Use a scalar variable with a unit, e.g. `sc.scalar(0.01, unit='meV')`, for the histogramming to get a constant bin width.

In [ ]:
folders = (
    "../3-mcstas/QENS_elastic_many_neutrons",
    "../3-mcstas/QENS_known_quasi_elastic_many_neutrons",
    "../3-mcstas/QENS_unknown_quasi_elastic_many_neutrons",
)

⚠️ **If you did not complete the Si simulation yesterday**,
you can use some pre-prepared data by uncommenting and running the cell below:

In [ ]:
# folders = (
#     utils.fetch_data("3-mcstas/QENS_elastic_many_neutrons"),
#     utils.fetch_data("3-mcstas/QENS_known_quasi_elastic_many_neutrons"),
#     utils.fetch_data("3-mcstas/QENS_unknown_quasi_elastic_many_neutrons")
# )

**Solution:**

In [ ]:
def process_sample(raw_events, bin_width=0.001 * sc.Unit('meV')):
    binned_for_mask = raw_events.bin(y=mask_regions)
    binned_for_mask.masks["bad_timing"] = mask

    in_energy_transfer = binned_for_mask.transform_coords(
        "energy_transfer", graph=graph
    )

    hist = in_energy_transfer.bins.concat().hist(energy_transfer=bin_width)

    return hist

results = {}
for folder in folders:
    events = utils.load_qens(folder)
    # Remove the file path prefix from the folder name
    data_name = folder.split("/")[-1]
    results[data_name] = process_sample(events)

pp.plot(results)

### Save result to disk

Finally, we need to save our results to disk so that the reduced data can be forwarded to the next step in the pipeline (data analysis).
We will use a simple text file for this.

This code assumes that you stored your histograms in a `dict` called `results`.
If this is not the case, you need to modify the code below.

In [ ]:
from scippneutron.io import save_xye

for name, result in results.items():
    # The simple file format does not support bin-edge coordinates.
    # So we convert to bin-centers first.
    data = result.copy()
    data.coords["energy_transfer"] = sc.midpoints(data.coords["energy_transfer"])

    save_xye(f"energy_transfer_{name}.dat", data)

## Bonus exercise

Re-run the reduction using the results from the simulations with less neutrons,
and compare the results.

**Solution:**

In [ ]:
bin_widths = [0.0001 * sc.Unit('meV')] + [0.001 * sc.Unit('meV')] * 2

figures = []
for bin_width, case in zip(bin_widths, folders):
    results = {}
    for folder in [case.replace("_many_neutrons", ""), case]:
        events = utils.load_qens(folder)
        data_name = str(folder).split("/")[-1]
        results[data_name] = process_sample(events, bin_width=bin_width)

    figures.append(pp.plot(results, title=case.replace("../3-mcstas/QENS_", "").replace("_many_neutrons", "")))

figures[0] + figures[1] + figures[2]